# torch.nn
`作者：Tina`
`时间：2018-05-11`

本文涉及内容：
- Parameters
- Containers（容器）
- [卷积层](#卷积层)
- [池化层](#池化层)
- [Padding Layer](#Padding层)
- [Non-linear Activations](#非线性激活函数)
- [Normalization layer](#归一化层)
- Recurrent Layer
- [Linear Layer](#线性层)
- [Dropout Layer](#Dropout)
- [Sparse Layer](#SparseLayer)
- [Distance Function](#DistanseFunction)
- [Loss Function](#LossFunction)
- [Vision Layer](#VisionLayer)
- [DataParallel layers](#Multi-GPU)
- [Utilities](#Utilities)

## Parameters
### class torch.nn.Parameter
一种模块参数张量。`Parameters`是`Tensor`的子类，`Paramenters`和`Modules`一起使用的时候会有一些特殊的属性，当Paramenters赋值给Module的属性的时候，他会自动的被加到`Module`的参数列表中，并且将出现在`parameters()`迭代器中。当赋值给一个Tensor时就不会有这样的影响，这是因为：有时我们可能想要缓存一些临时状态，如果没有`Parameter`这个类的话，那么这些临时变量也会注册成为模型变量。

参数说明：
- data：参数张量
- requires_grad(bool,optional):默认是True。

## 容器(Containers)
包含以下内容：
- Module
- Sequential
### class torch.nn.Module
所有神经网络模块的基类。你的模型也应该继承这个类。模块还可以包含其他模块，允许将它们嵌套在树形结构中。你可以将子模块赋值给模型属性。

In [1]:
import torch.nn as nn
import torch.nn.functional as F

class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, 5)
        self.conv2 = nn.Conv2d(20, 20, 5)

    def forward(self, x):
       x = F.relu(self.conv1(x))
       return F.relu(self.conv2(x))

通过上面方式赋值的`submodule`会被注册。当调用 `.cuda()`的时候，`submodule`的参数也会转换为`cuda Tensor`。
#### add_module(name, module)
将一个子模块添加到当前模块中。被添加的`module`可以通过`name`属性来获取。
参数说明：
- name(string)：child module的名字。
- module：子模块被添加到这个模块中
例如：

In [2]:
import torch.nn as nn
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.add_module("conv", nn.Conv2d(10, 20, 4))
        #self.conv = nn.Conv2d(10, 20, 4) 和上面这个增加module的方式等价
model = Model()
print(model.conv)

Conv2d(10, 20, kernel_size=(4, 4), stride=(1, 1))


#### apply(fn)
将fn递归地应用到每一个子模块（由.children()返回）以及self中。典型的使用包括初始化模型的参数。参见`torch.nn.init`。
参数说明：
- fn：Module->none，应用到每个子模块的函数
- 返回：self，类型是module

In [3]:
def init_weights(m):
        print(m)
        if type(m) == nn.Linear:
            m.weight.data.fill_(1.0)
            print(m.weight)

net = nn.Sequential(nn.Linear(2, 2), nn.Linear(2, 2))
net.apply(init_weights) # 将init_weights应用到每一个子模块中

Linear(in_features=2, out_features=2, bias=True)
Parameter containing:
tensor([[ 1.,  1.],
        [ 1.,  1.]])
Linear(in_features=2, out_features=2, bias=True)
Parameter containing:
tensor([[ 1.,  1.],
        [ 1.,  1.]])
Sequential(
  (0): Linear(in_features=2, out_features=2, bias=True)
  (1): Linear(in_features=2, out_features=2, bias=True)
)


Sequential(
  (0): Linear(in_features=2, out_features=2, bias=True)
  (1): Linear(in_features=2, out_features=2, bias=True)
)

#### children()
返回当前模型子模块的迭代器。

In [4]:
import torch.nn as nn
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.add_module("conv", nn.Conv2d(10, 20, 4))
        self.add_module("conv1", nn.Conv2d(20 ,10, 4))
model = Model()

for sub_module in model.children():
    print(sub_module)

Conv2d(10, 20, kernel_size=(4, 4), stride=(1, 1))
Conv2d(20, 10, kernel_size=(4, 4), stride=(1, 1))


#### cpu()
将所有模型参数和缓冲区复制到CPU。

#### cuda(device=None)
将所有的模型参数(parameters)和buffersf复制给GPU。

这也把相关的参数和缓冲变成不同的对象。因此，如果模块在优化时使用GPU，在构建优化器之前应该先调用它。
#### double()
将所有浮点参数和缓冲区都转换为double类型。

#### dump_patches = False
这使得BC更好地支持`load_state_dict()`。在`state_dict`中，版本号将被保存在返回的`state_dict`的`attribute _metadata`中。因此，`pickled. _metadata`是一个字典，`key`跟随`state_dict`的命名公约。

如果新的参数/缓冲区从一个模块中被添加/删除，这个数字将会变化，并且模块的`_load_from_state_dict`方法会比较版本号，并且如果`state dict`来自于之前的改变，还会做适当的修改。

#### eval()
将模块设置为评估模式。这只对某些模块有影响，当模型中有`Dropout`和`BatchNorm`等就会有影响。
#### extra_repr()
设置模块的额外表示。

打印自定义的额外信息，你应该在你自己的模块中实现这个方法。单行和多行字符串都是可以接受的。
#### float()
将`parameters`和`buffers`的数据类型转换成`float`。
#### forward(*input)
定义了每次执行的计算步骤。在所有的子类中都需要重写这个函数。
#### half()
将`parameters`和`buffers`的数据类型转换成`half`。
#### load_state_dict(state_dict, strict=True)
将`state_dict`中的`parameters`和`buffers`复制到此模块和它的子后代中。如果`strict`为`True`， 则`state_dict`的`key`必须和模块的`state_dict()`函数返回的`key`一致。用来加载模型参数。

参数说明：
- state_dict (dict) – 一个包含 parameters 和 persistent buffers（持久化缓存的）字典
- strict (bool) – 严格的强制 state_dict 属性中的 key 与该模块的函数的 state_dict() 返回的 keys 相匹配。
#### modules()
返回一个包含当前模型所有模块的迭代器。重复的模块只返回一次。在下面的例子中，l只返回一次。

In [5]:
l = nn.Linear(2, 2)
net = nn.Sequential(l, l)
for idx, m in enumerate(net.modules()):
    print(idx, '->', m)

0 -> Sequential(
  (0): Linear(in_features=2, out_features=2, bias=True)
  (1): Linear(in_features=2, out_features=2, bias=True)
)
1 -> Linear(in_features=2, out_features=2, bias=True)


#### named_children()
返回模型当前子模块的迭代器，会产生模块的名称和模块本身

In [6]:
for name, module in model.named_children():
    if name in ['conv4', 'conv5']:
        print(module)

#### named_modules(memo=None, prefix='')
返回网络中所有模块的迭代器，会产生模块的名称和模块本身。重复模块只返回一次。下面的例子中，1将只返回一次。

In [7]:
l = nn.Linear(2, 2)
net = nn.Sequential(l, l)
for idx, m in enumerate(net.named_modules()):
        print(idx, '->', m)

0 -> ('', Sequential(
  (0): Linear(in_features=2, out_features=2, bias=True)
  (1): Linear(in_features=2, out_features=2, bias=True)
))
1 -> ('0', Linear(in_features=2, out_features=2, bias=True))


#### named_parameters(memo=None, prefix='')
返回模块参数的迭代器, 产生参数的名称以及参数本身。
```
for name, param in self.named_parameters():
    if name in ['bias']:
        print(param.size())
```

#### parameters()
返回一个模块参数的迭代器。这通常传递给优化器。

In [8]:
for param in model.parameters():
    print(type(param.data), param.size())

<class 'torch.Tensor'> torch.Size([20, 10, 4, 4])
<class 'torch.Tensor'> torch.Size([20])
<class 'torch.Tensor'> torch.Size([10, 20, 4, 4])
<class 'torch.Tensor'> torch.Size([10])


#### register_backward_hook(hook)
在module上注册一个bachward hook。每次计算module的inputs的梯度的时候，这个hook会被调用。hook应该有以下结构：
```python
hook(module, grad_input, grad_output) -> Tensor or None
```
如果module有多个输入输出的话，那么grad_input和grad_output将会是元组。 hook不应该修改它的arguments，但是它可以选择性的返回关于输入的梯度，这个返回的梯度在后续的计算中会替代grad_input。
#### register_buffer(name, tensor)
向模块添加持久缓冲区。这通常用于注册一个不应该被认为是模型参数的缓冲区，比如`BatchNorm`的`running_maen`不是一个参数，而是持续状态的一部分。
缓冲区可以用给定名字访问。
参数说明：
- name：buffer的名字，是一个string
- tensor：被注册的buffer
```python
 self.register_buffer('running_mean', torch.zeros(num_features))
```

#### register_forward_hook(hook)
在module上注册一个forward hook。 每次调用forward()计算输出的时候，这个hook就会被调用。它应该拥有以下结构：
```python
hook(module, input, output) -> None
```
hook不应该修改 input和output的值。 这个函数返回一个 句柄(handle)。它有一个方法  handle.remove()，可以用这个方法将hook从module移除
#### register_forward_pre_hook(hook)
在module上注册一个forward pre-hook。这个hook在`forward()`被调用之前调用。应该有以下结构：
```python
hook(module, input) -> None
```
这个hook不应该修改input，返回一个句柄，可以使用`handle.remove`从module中移除。

#### register_parameter(name, param)
向模块添加一个参数。可以使用指定的 name 属性来访问参数。其中`param`是要被添加到模块的参数。
#### state_dict(destination=None, prefix='', keep_vars=False)
返回一个字典，保存着module的所有状态（state）。包括参数和持久化的缓冲区 (例如运行中的平均值)。`Key`是与之对应的参数和缓冲区的`name`。

In [9]:
module.state_dict().keys()

odict_keys(['weight', 'bias'])

#### to(*args, **kwargs)
移动和/或转换参数和缓冲区。调用方式：
- to(device)
- to(dtype)
- to(device,dtype)
它与`torch.Tensor.to()`有着相似的调用方式，但是不是接受一个Tensor，而是接受一个浮点类型`dtype`。这个方法以原地形式修改了模块。
参数说明：
- device：参数和缓冲区将要被复制到这个device上
- dtype：这个模块中浮点类型参数和缓冲区将要被转换的期望浮点类型
返回seft，类型为module

In [10]:
import torch
linear = nn.Linear(2, 2)
print(linear.weight)

linear.to(torch.double)
print(linear.weight)

gpu1 = torch.device("cuda:1")
linear.to(gpu1, dtype=torch.half)
print(linear.weight)

cpu = torch.device("cpu")
linear.to(cpu)
print(linear.weight)

Parameter containing:
tensor([[-0.5648, -0.6641],
        [-0.0987, -0.6665]])
Parameter containing:
tensor([[-0.5648, -0.6641],
        [-0.0987, -0.6665]], dtype=torch.float64)
Parameter containing:
tensor([[-0.5649, -0.6641],
        [-0.0987, -0.6665]], dtype=torch.float16, device='cuda:1')
Parameter containing:
tensor([[-0.5649, -0.6641],
        [-0.0987, -0.6665]], dtype=torch.float16)


#### train(mode=True)
把模块设置为训练模式。这只对某些模块有影响，比如`Dropout`和`BatchNorm`
#### type(dst_type)
将所有参数和缓冲区转换为`dst_type`。
#### zero_grad()
设置模型参数的梯度为0。

### class torch.nn.Sequential(*args)
一个顺序容器。模块将按照它们在构造器中传递的顺序添加到其中。或者，也可以传递进来一个模块的有序字典。

```python
# Example of using Sequential
model = nn.Sequential(
          nn.Conv2d(1,20,5),
          nn.ReLU(),
          nn.Conv2d(20,64,5),
          nn.ReLU()
        )

# Example of using Sequential with OrderedDict
model = nn.Sequential(OrderedDict([
          ('conv1', nn.Conv2d(1,20,5)),
          ('relu1', nn.ReLU()),
          ('conv2', nn.Conv2d(20,64,5)),
          ('relu2', nn.ReLU())
        ]))
```

### class torch.nn.ModuleList(modules=None)
将子模块保存在一个列表中。ModuleList可以像普通的Python列表一样被索引，但是它所包含的模块是正确注册的，并且将被所有的模块方法看到。
```python
class MyModule(nn.Module):
    def __init__(self):
        super(MyModule, self).__init__()
        self.linears = nn.ModuleList([nn.Linear(10, 10) for i in range(10)]) #在ModuleList中保存了10个nn.Linear(10,10)

    def forward(self, x):
        # ModuleList can act as an iterable, or be indexed using ints
        for i, l in enumerate(self.linears):
            x = self.linears[i // 2](x) + l(x)
        return x
```
#### append(module)
将一个给定的模块附加到列表的末尾。
#### extend(modules)
把Python迭代器中的一个模块添加到列表末尾。

### class torch.nn.ParameterList(parameters=None)
将参数保存在列表中。参数列表可以像普通的Python列表一样被编入索引，但是它包含的参数被正确地注册，并且所有模块方法都可以看到它。
```python
class MyModule(nn.Module):
    def __init__(self):
        super(MyModule, self).__init__()
        self.params = nn.ParameterList([nn.Parameter(torch.randn(10, 10)) for i in range(10)])

    def forward(self, x):
        # ParameterList can act as an iterable, or be indexed using ints
        for i, p in enumerate(self.params):
            x = self.params[i // 2].mm(x) + p.mm(x)
        return x
```
#### append(parameter)
在列表的末尾添加一个给定的参数。
#### extend(parameters)
在列表最后添加python迭代器中的参数

##  <div id="卷积层"></div>卷积层
包含内容：
- Conv1d
- Conv2d
- Conv3d
- ConvTranspose1d
- ConvTranspose2d
- ConvTranspose3d


### class torch.nn.Conv1d(in_channels, out_channels, kernel_size, stride=1, padding=0, dilation=1, groups=1, bias=True)
一维卷积层，输入的尺度是$(N, C_in, L)$，输出尺度$(N,C_out,L_out)$：$$ out(N_i, C_{out_j})=bias(C {out_j})+\sum^{C{in}-1}_{k=0}weight(C{out_j},k) ⋆ input(N_i,k) $$
- ⋆是卷积运算符, 上式带 ⋆ 项为卷积项。
- N是Batch Size，L是信号序列的长度
参数说明：
- in_channels(int)：输入信号的通道
- out_channels(int)：卷积产生的通道
- kerner_size(int or tuple)：卷积核的尺寸
- stride: 控制相关系数的计算步长 
- dilation: 用于控制卷积核点之间的距离，详细描述在[这里](https://github.com/vdumoulin/conv_arithmetic/blob/master/README.md) 
- groups: 控制输入和输出之间的连接，group=1，输出是所有的输入的卷积；group=2，此时相当于有并列的两个卷积层，每个卷积层计算输入通道的一半，并且产生的输出是输出通道的一半，随后将这两个输出连接起来；group=in_channels，每个输入通道与它自己的卷积核集合(size是$\frac {out_channels} {in_channels}$的向下取整)进行卷积
- bias(bool, optional)：如果bias=True，添加偏置

注意：数据的最后一列可能会因为 kernal 大小设定不当而被丢弃（大部分发生在 kernal 大小不能被输入整除的时候, 适当的 padding 可以避免这个问题）。
**shape**:
- 输入: $(N,C_in,L_in)$
- 输出: $(N,C_out,L_out)$

输入输出的计算方式： 
$$L_{out}=floor(\frac {L_{in}+2 \times padding-dilation(kernerlSize-1)-1} {stride}+1)$$
**变量**：
- weight(tensor) - 卷积的权重，大小是(out_channels, in_channels, kernel_size) 
- bias(tensor) - 卷积的偏置系数，大小是（out_channel）

In [11]:
m = nn.Conv1d(16, 33, 3, stride=2)
input = torch.randn(20, 16, 50)
output = m(input)

### class torch.nn.Conv2d(in_channels, out_channels, kernel_size, stride=1, padding=0, dilation=1, groups=1, bias=True)
二维卷积层, 输入的尺度是$(N, C_in,H,W)$，输出尺度$(N,C_{out},H_{out},W_{out})$。

**shape**:
- input: $(N,C_{in},H_{in},W_{in})$ 
- output: $(N,C_{out},H_{out},W_{out})$
$$H_{out}=floor(\frac {H_{in}+2 \times padding[0]-dilation[0](kernerlSize[0]-1)-1)} {stride[0]}+1)$$

$$W_{out}=floor(\frac {W_{in}+2 \times padding[1]-dilation[1](kernerlSize[1]-1)-1)} {stride[1]}+1)$$

**变量**:
- weight(tensor) - 卷积的权重，大小是(out_channels, in_channels,kernel_size[0], kernel_size[1])
- bias(tensor) - 卷积的偏置系数，大小是（out_channel）

In [12]:
# With square kernels and equal stride
m = nn.Conv2d(16, 33, 3, stride=2)
# non-square kernels and unequal stride and with padding
m = nn.Conv2d(16, 33, (3, 5), stride=(2, 1), padding=(4, 2))
# non-square kernels and unequal stride and with padding and dilation
m = nn.Conv2d(16, 33, (3, 5), stride=(2, 1), padding=(4, 2), dilation=(3, 1))
input = torch.randn(20, 16, 50, 100)
output = m(input)

### class torch.nn.Conv3d(in_channels, out_channels, kernel_size, stride=1, padding=0, dilation=1, groups=1, bias=True)
三维卷积层, 输入的尺度是$(N, C_{in},D,H,W)$，输出尺度$(N,C_out,D_{out},H_{out},W_{out})$的计算方式：
**shape**:
- input: $(N,C_{in},D_{in},H_{in},W_{in})$ 
- output: (N,C_{out},D_{out},H_{out},W_{out}) 
$$D_{out}=floor(\frac {D_{in}+2 \times padding[0]-dilation[0](kernerlSize[0]-1)-1} {stride[0]}+1)$$

$$H_{out}=floor(\frac {H_{in}+2 \times padding[1]-dilation[2](kernerlSize[1]-1)-1} {stride[1]}+1)$$

$$W_{out}=floor(\frac {W_{in}+2 \times padding[2]-dilation[2](kernerlSize[2]-1)-1} {stride[2]}+1)$$

**变量*:
weight(tensor) - 卷积的权重，shape是(out_channels, in_channels,kernel_size[0], kernel_size[1], kernel_size[2])
bias(tensor) - 卷积的偏置系数，shape是（out_channel）

In [13]:
# With square kernels and equal stride
m = nn.Conv3d(16, 33, 3, stride=2)
# non-square kernels and unequal stride and with padding
m = nn.Conv3d(16, 33, (3, 5, 2), stride=(2, 1, 1), padding=(4, 2, 0))
input = torch.randn(20, 16, 10, 50, 100)
output = m(input)

### class torch.nn.ConvTranspose1d(in_channels, out_channels, kernel_size, stride=1, padding=0, output_padding=0, groups=1, bias=True, dilation=1)
一维转置卷积操作。这个模块可以被看作是Conv1d关于输入的梯度。有时候也被称为反卷积(deconvolution)，尽管它并不是真正的反卷积操作。
因为输入和输出是不是完全的互相关。因此，用户可以进行适当的填充（padding操作）。
参数说明：
- output_padding：输出的每一条边补充0的数

**shape**: 
- 输入: $(N,C_{in},L_{in})$ 
- 输出: $(N,C_{out},L_{out})$
$$L_{out}=(L_{in}-1)stride-2 \times padding+kernelSize+outputPadding$$

### class torch.nn.ConvTranspose2d(in_channels, out_channels, kernel_size, stride=1, padding=0, output_padding=0, groups=1, bias=True, dilation=1)
2维的转置卷积操作。与一维类比。
**shape**: 
- 输入: $(N,C_{in},H_{in}，W_{in})
- 输出: $(N,C_{out},H_{out},W_{out})$
$$H_{out}=(H_{in}-1) \times stride[0]-2 \times padding[0]+kernelSize[0]+outputPadding[0]$$

$$W_{out}=(W_{in}-1) \times stride[1]-2 \times padding[1]+kernelSize[1]+outputPadding[1]$$

变量: 
- weight(tensor) - 卷积的权重，大小是(in_channels, in_channels,kernel_size[0], kernel_size[1]) 
- bias(tensor) - 卷积的偏置系数，大小是（out_channel）

In [14]:
# With square kernels and equal stride
m = nn.ConvTranspose2d(16, 33, 3, stride=2)
# non-square kernels and unequal stride and with padding
m = nn.ConvTranspose2d(16, 33, (3, 5), stride=(2, 1), padding=(4, 2))
input = torch.randn(20, 16, 50, 100)
output = m(input)
print(output.size())

# exact output size can be also specified as an argument
input = torch.randn(1, 16, 12, 12)
downsample = nn.Conv2d(16, 16, 3, stride=2, padding=1)
upsample = nn.ConvTranspose2d(16, 16, 3, stride=2, padding=1)
h = downsample(input)
print(h.size())
output = upsample(h, output_size=input.size())
print(output.size())

torch.Size([20, 33, 93, 100])
torch.Size([1, 16, 6, 6])
torch.Size([1, 16, 12, 12])


### class torch.nn.ConvTranspose3d(in_channels, out_channels, kernel_size, stride=1, padding=0, output_padding=0, groups=1, bias=True, dilation=1)
与上面的类比。

## <div id="池化层"></div>池化层
### class torch.nn.MaxPool1d(kernel_size, stride=None, padding=0, dilation=1, return_indices=False, ceil_mode=False)
对于输入信号的输入通道，进行1维最大池化（max pooling）操作。如果输入的大小是$(N,C,L)$，那么输出的大小是$(N,C,L_out)$的计算方式是： 
$$out(N_i, C_j,k)=max_{m=0,...kernel_size-1}input(N_{i},C_j,stride*k+m)$$

参数说明：
- kernel_size(int or tuple) - max pooling的窗口大小
- stride(int or tuple, optional) - max pooling的窗口移动的步长。默认值是kernel_size
- padding(int or tuple, optional) - 输入的每一条边补充0的数
- dilation(int or tuple, optional) – 一个控制窗口中元素步幅的参数
- return_indices - 如果等于True，会返回输出最大值的序号，对于上采样操作会有帮助
- ceil_mode - 如果等于True，计算输出信号大小的时候，会使用向上取整，代替默认的向下取整的操作

**shape**:
- 输入: $(N,C_{in},L_{in})$ 
- 输出: $(N,C_{out},L_{out})$ 
$$L_{out}=floor(\frac {L_{in} + 2 \times padding - dilation(kernel_size - 1) - 1} {stride} + 1$$

In [15]:
# pool of size=3, stride=2
m = nn.MaxPool1d(3, stride=2)
input = torch.randn(20, 16, 50)
output = m(input)

### class torch.nn.MaxPool2d(kernel_size, stride=None, padding=0, dilation=1, return_indices=False, ceil_mode=False)
二维最大池化操作。如果输入的大小是$(N,C,H,W)$，那么输出的大小$(N,C,H_{out},W_{out})$和池化窗口大小$(kH,kW)$的关系是： 
$$out(N_i, C_j,k)=max_{m=0,...kH-1}max_{m=0,...kW-1}input(N_{i},C_j,stride[0]*h+m,stride[1]*w+n)$$

shape: 
输入: (N,C,H_{in},W_in) 
输出: (N,C,H_out,W_out) 
$$H_{out}=floor(\frac {H_{in} + 2padding[0] - dilation[0](kernel_size[0] - 1) - 1} {stride[0]} + 1$$

$$W_{out}=floor(\frac {W_{in} + 2padding[1] - dilation[1](kernel_size[1] - 1) - 1)} {stride[1]} + 1$$

In [16]:
# pool of square window of size=3, stride=2
m = nn.MaxPool2d(3, stride=2)
# pool of non-square window
m = nn.MaxPool2d((3, 2), stride=(2, 1))
input = torch.randn(20, 16, 50, 32)
output = m(input)

### class torch.nn.MaxPool3d(kernel_size, stride=None, padding=0, dilation=1, return_indices=False, ceil_mode=False)
同上面的类似。
### class torch.nn.MaxUnpool1d(kernel_size, stride=None, padding=0)
Maxpool1d的逆过程，不过并不是完全的逆过程，因为在maxpool1d的过程中，一些最大值的已经丢失。 MaxUnpool1d输入MaxPool1d的输出，包括最大值的索引，并计算所有maxpool1d过程中非最大值被设置为零的部分的逆。

注意：
MaxPool1d可以将多个输入大小映射到相同的输出大小。因此，逆过程可能会变得模棱两可。为了适应这一点，可以在调用中将输出大小（output_size）作为额外的参数传入。 具体用法，请参阅下面的输入和示例。

输入：
- input:需要转换的input 
- tensor 索引：由Maxpool1d给出
- output_size:一个指定输出大小的torch.Size

**shape**:
- input: $(N,C,H_{in})$
- output:$(N,C,H_{out})$ 
$$H_{out}=(H_{in}-1)stride[0]-2padding[0]+kernelSize[0]$$ 
也可以使用output_size指定输出的大小

In [17]:
pool = nn.MaxPool1d(2, stride=2, return_indices=True)
unpool = nn.MaxUnpool1d(2, stride=2)
input = torch.tensor([[[1., 2, 3, 4, 5, 6, 7, 8]]])
output, indices = pool(input)
print(output)
print(unpool(output, indices))

# Example showcasing the use of output_size
input = torch.tensor([[[1., 2, 3, 4, 5, 6, 7, 8, 9]]])
output, indices = pool(input)
print(unpool(output, indices, output_size=input.size()))

print(unpool(output, indices))

tensor([[[ 2.,  4.,  6.,  8.]]])
tensor([[[ 0.,  2.,  0.,  4.,  0.,  6.,  0.,  8.]]])
tensor([[[ 0.,  2.,  0.,  4.,  0.,  6.,  0.,  8.,  0.]]])
tensor([[[ 0.,  2.,  0.,  4.,  0.,  6.,  0.,  8.]]])


### class torch.nn.MaxUnpool2d(kernel_size, stride=None, padding=0)
MaxUnpool2d的输入是MaxPool2d的输出，包括最大值的索引，并计算所有maxpool2d过程中非最大值被设置为零的部分的逆。

In [18]:
pool = nn.MaxPool2d(2, stride=2, return_indices=True)
unpool = nn.MaxUnpool2d(2, stride=2)
input = torch.Tensor([[[[ 1,  2,  3,  4],
                        [ 5,  6,  7,  8],
                        [ 9, 10, 11, 12],
                        [13, 14, 15, 16]]]])
output, indices = pool(input)
print(unpool(output, indices))
# specify a different output size than input siz
print(unpool(output, indices, output_size=torch.Size([1, 1, 5, 5])))


tensor([[[[  0.,   0.,   0.,   0.],
          [  0.,   6.,   0.,   8.],
          [  0.,   0.,   0.,   0.],
          [  0.,  14.,   0.,  16.]]]])
tensor([[[[  0.,   0.,   0.,   0.,   0.],
          [  6.,   0.,   8.,   0.,   0.],
          [  0.,   0.,   0.,  14.,   0.],
          [ 16.,   0.,   0.,   0.,   0.],
          [  0.,   0.,   0.,   0.,   0.]]]])


### class torch.nn.MaxUnpool3d(kernel_size, stride=None, padding=0)
与上面类比。
### class torch.nn.AvgPool1d(kernel_size, stride=None, padding=0, ceil_mode=False, count_include_pad=True)
一维平均池化。

In [19]:
# pool with window of size=3, stride=2
m = nn.AvgPool1d(3, stride=2)
print(m(torch.tensor([[[1.,2,3,4,5,6,7]]]))
)

tensor([[[ 2.,  4.,  6.]]])


### class torch.nn.AvgPool2d(kernel_size, stride=None, padding=0, ceil_mode=False, count_include_pad=True)
其中的参数：
- count_include_pad：如果等于True，计算平均池化时，将包括padding填充的0。

In [20]:
# pool of square window of size=3, stride=2
m = nn.AvgPool2d(3, stride=2)
# pool of non-square window
m = nn.AvgPool2d((3, 2), stride=(2, 1))
input = torch.randn(20, 16, 50, 32)
output = m(input)

### class torch.nn.AvgPool3d(kernel_size, stride=None, padding=0, ceil_mode=False, count_include_pad=True)
同上类似。
### class torch.nn.FractionalMaxPool2d(kernel_size, output_size=None, output_ratio=None, return_indices=False, _random_samples=None)
二维的[分数最大化池化](https://arxiv.org/abs/1412.6071)。
### 更多池化操作
参加[全部的池化操作](https://pytorch.org/docs/stable/nn.html#pooling-layers)

## <div id="Padding层"></div> Padding层
### class torch.nn.ReflectionPad1d(padding)
使用输入边界的反射填充输入张量。
- padding (int, tuple) – 填充的大小。如果是int, 则在所有边界填充使用相同的。如果是元组的话，对于一维的就是左右分别填充的数量。

In [21]:
m = nn.ReflectionPad1d(2) #也就是各个边都填充2个数字，数字内容就是input的前两列的倒序
input = torch.arange(8).reshape(1, 2, 4)
print(input)
print(m(input))

# using different paddings
m = nn.ReflectionPad1d((3, 1)) #左右分别填充3个和1个数
print(m(input))

tensor([[[ 0.,  1.,  2.,  3.],
         [ 4.,  5.,  6.,  7.]]])
tensor([[[ 2.,  1.,  0.,  1.,  2.,  3.,  2.,  1.],
         [ 6.,  5.,  4.,  5.,  6.,  7.,  6.,  5.]]])
tensor([[[ 3.,  2.,  1.,  0.,  1.,  2.,  3.,  2.],
         [ 7.,  6.,  5.,  4.,  5.,  6.,  7.,  6.]]])


### class torch.nn.ReflectionPad2d(padding)
参数`padding`，如果是int，所有边界填充大小一样；如果是4元组，则表示上下左右填充的数目（paddingLeft, paddingRight, paddingTop, paddingBottom）。

In [22]:
m = nn.ReflectionPad2d(2)
input = torch.arange(9).reshape(1, 1, 3, 3) 
print(input)
print(m(input))

# using different paddings
m = nn.ReflectionPad2d((1, 1, 2, 0))
print(m(input))

tensor([[[[ 0.,  1.,  2.],
          [ 3.,  4.,  5.],
          [ 6.,  7.,  8.]]]])
tensor([[[[ 8.,  7.,  6.,  7.,  8.,  7.,  6.],
          [ 5.,  4.,  3.,  4.,  5.,  4.,  3.],
          [ 2.,  1.,  0.,  1.,  2.,  1.,  0.],
          [ 5.,  4.,  3.,  4.,  5.,  4.,  3.],
          [ 8.,  7.,  6.,  7.,  8.,  7.,  6.],
          [ 5.,  4.,  3.,  4.,  5.,  4.,  3.],
          [ 2.,  1.,  0.,  1.,  2.,  1.,  0.]]]])
tensor([[[[ 7.,  6.,  7.,  8.,  7.],
          [ 4.,  3.,  4.,  5.,  4.],
          [ 1.,  0.,  1.,  2.,  1.],
          [ 4.,  3.,  4.,  5.,  4.],
          [ 7.,  6.,  7.,  8.,  7.]]]])


### class torch.nn.ReplicationPad1d(padding)
使用输入边界的复制填充输入张量。
### class torch.nn.ReplicationPad2d(padding)
### class torch.nn.ReplicationPad3d(padding)

In [23]:
m = nn.ReplicationPad2d(2)
input = torch.arange(9).reshape(1, 1, 3, 3)
print(input)
print(m(input))

# using different paddings
m = nn.ReplicationPad2d((1, 1, 2, 0))
print(m(input))

tensor([[[[ 0.,  1.,  2.],
          [ 3.,  4.,  5.],
          [ 6.,  7.,  8.]]]])
tensor([[[[ 0.,  0.,  0.,  1.,  2.,  2.,  2.],
          [ 0.,  0.,  0.,  1.,  2.,  2.,  2.],
          [ 0.,  0.,  0.,  1.,  2.,  2.,  2.],
          [ 3.,  3.,  3.,  4.,  5.,  5.,  5.],
          [ 6.,  6.,  6.,  7.,  8.,  8.,  8.],
          [ 6.,  6.,  6.,  7.,  8.,  8.,  8.],
          [ 6.,  6.,  6.,  7.,  8.,  8.,  8.]]]])
tensor([[[[ 0.,  0.,  1.,  2.,  2.],
          [ 0.,  0.,  1.,  2.,  2.],
          [ 0.,  0.,  1.,  2.,  2.],
          [ 3.,  3.,  4.,  5.,  5.],
          [ 6.,  6.,  7.,  8.,  8.]]]])


### class torch.nn.ZeroPad2d(padding)
用零填充输入张量边界。

In [25]:
m = nn.ZeroPad2d(2)
input = torch.randn(1, 1, 3, 3)
print(input)
print(m(input))
# using different paddings
m = nn.ZeroPad2d((1, 1, 2, 0)) #左，右，上，下填充的数目
print(m(input))

tensor([[[[-0.2384, -0.3068, -1.1337],
          [-0.7758,  0.6120,  0.3402],
          [ 0.3969,  1.0994,  0.8858]]]])
tensor([[[[ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000, -0.2384, -0.3068, -1.1337,  0.0000,  0.0000],
          [ 0.0000,  0.0000, -0.7758,  0.6120,  0.3402,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.3969,  1.0994,  0.8858,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000]]]])
tensor([[[[ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
          [ 0.0000, -0.2384, -0.3068, -1.1337,  0.0000],
          [ 0.0000, -0.7758,  0.6120,  0.3402,  0.0000],
          [ 0.0000,  0.3969,  1.0994,  0.8858,  0.0000]]]])


### class torch.nn.ConstantPad1d(padding, value)
### class torch.nn.ConstantPad2d(padding, value)
### class torch.nn.ConstantPad3d(padding, value)
用一个常数值填充输入张量边界。

## <div id = "非线性激活函数"></div>Non-linear activations (weighted sum, nonlinearity)
### class torch.nn.ELU(alpha=1.0, inplace=False)
逐元素的应用ELU函数。$ELU(x) = \max(0,x) + min(0, \alpha * (exp(x) - 1))$

Parameters:	
- alpha – ELU 定义公式中的 alpha 值. 默认值: 1.0
- inplace – 选择是否进行覆盖运算 默认值: False
### class torch.nn.LeakyReLU(negative_slope=0.01, inplace=False)
### class torch.nn.PReLU(num_parameters=1, init=0.25)
### class torch.nn.ReLU(inplace=False)
### class torch.nn.Sigmoid 
- Input: (N,∗) * 表示任意维度组合
- Output: (N,∗), 与输入有相同的 shape 属性
### class torch.nn.Tanh
### class torch.nn.Softmax(dim=None)
### class torch.nn.Softmax2d
对特征图的每一个位置应用softmax
Shape:
- Input: (N,C,H,W)
- Output: (N,C,H,W) (格式 shape 与输入相同)


## <div id = "归一化层"></div>Normalization layers
- class torch.nn.BatchNorm1d(num_features, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
- class torch.nn.BatchNorm2d(num_features, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)：小批量 (mini-batch) 数据进行批标准化 (Batch Normalization) 操作。$$y = \frac {x - E[x]} { \sqrt{Var[x] + \epsilon}} * \gamma + \beta$$ 每个小批量数据中,计算各个维度的均值和标准差。并且$\gamma$和$\beta$是大小为 C 的可学习参数向量( C 为输入大小)。在训练过程中，该层持续的计算均值和方差的估计，在预测时使用这些值即可，默认的`momentum`为 0.1。

  参数：
  - num_features：从输入尺寸中得到C作为num_features。inoput:(N,C,H.W)
  - affine：布尔值,设为 True 时,表示该层添加可学习,可改变的仿射参数即 gamma 和 beta。
  - momentum： 动态均值和动态方差使用的移动动量值,默认为 0.1。

  Shape:
  - Input: (N,C) or (N,C,L)
  - Output: (N,C) or (N,C,L) (same shape as input)

In [26]:
# With Learnable Parameters
m = nn.BatchNorm2d(100)
# Without Learnable Parameters
m = nn.BatchNorm2d(100, affine=False)
input = torch.randn(20, 100, 35, 45)
output = m(input)

## Recurrent Layer
待补充

## <div id = "线性层"></div>Linear Layer
### class torch.nn.Linear(in_features, out_features, bias=True)
对输入数据做线性变换，$y = Ax+b$

**参数**
- in_features - 每个输入样本的大小
- out_features - 每个输出样本的大小
- bias - 若设置为False，该层不会学习偏置。默认值：True

**Shape**
- 输入: $(N,in_features)$
- 输出：$(N,out_features)$
**变量**
- weight -形状为$(OutFeatures * InFeatures)$的模块中可学习的权值
- bias -形状为$(OutFeatures)$的模块中可学习的偏置

In [27]:
m = nn.Linear(20, 30)
input = torch.randn(128, 20)
output = m(input)
print(output.size())

torch.Size([128, 30])


### class torch.nn.Bilinear(in1_features, in2_features, out_features, bias=True)
对输入数据进行双线性变换。$y = x_1 * A * x_2 + b$

**参数**
- in1_features – 输入一的每个输入样本的大小
- in2_features – 输入二的每个输入样本的大小
- out_features – 每个输出样本的大小
- bias – 若设置为False，这层不会学习偏置。默认值: True

**Shape**
- Input: $(N,*,In1Features), (N,*,In2Features)$，*表示任意数量的额外维度，除了最后一维，所有的维度应该是相同的
- Output: $(N,*,OutFeatures)$除了最后一维，所有的维度与输入相同

**变量**
- weight – 形状为 $(OutFeatures * In1Features * In2Features)$ 的模块中可学习的权值
- bias – 形状为 $(OutFeatures)$ 的模块中可学习的偏置

In [28]:
m = nn.Bilinear(20, 30, 40)
input1 = torch.randn(128, 20)
input2 = torch.randn(128, 30)
output = m(input1, input2)
print(output.size())

torch.Size([128, 40])


## <div id= "Dropout"></div>Dropout layers
### class torch.nn.Dropout(p=0.5, inplace=False)
在训练期间, 按照伯努利概率分布, 以概率 p 随机地将输入张量中的部分元素设置为0。在每次前向传播调用时, 被置为 0 的元素是随机的。

**参数**
- p - 将元素置0的概率。默认值：0.5
- in-place - 若设置为True，会在原地执行操作。默认值：False

**形状**
- 输入： 任意。输入可以为任意形状。
- 输出： 相同。输出和输入形状相同。

在训练中，Dropout的输出通过因子$\frac 1 {1-p}$进行缩放，保证了在预测时计算的是一个恒等函数。

In [29]:
m = nn.Dropout(p=0.2)
input = torch.randn(20, 16)
output = m(input)

### class torch.nn.Dropout2d(p=0.5, inplace=False)
将输入张量的所有通道随机地置为 0。在每次调用时被置为 0 的通道是随机的。通常输入数据来自 Conv2d 模块。

论文[Efficient Object Localization Using Convolutional Networks](https://arxiv.org/abs/1411.4280)中提到：如果feature map中的相邻像素是强相关的（在前几层卷积层中很常见），那么独立同分布 的 dropout 将不会正则化激活函数, 相反其会有效地降低学习率。在这样的情况下, 应该使用函数 `nn.Dropout2d` , 它能够提升feature map之间的独立性。

**参数**
- p(float, optional) - 将元素置0的概率。
- inplace(bool, optional) - 若设置为True，会在原地执行操作。

**形状**
- 输入： (N,C,H,W)
- 输出： (N,C,H,W)（与输入形状相同）

In [32]:
m = nn.Dropout2d(p=0.2)
input = torch.randn(20, 16, 32, 32)
output = m(input)
print(output)

tensor([[[[-1.5373, -0.3664, -0.4707,  ...,  1.1798, -0.9644, -1.0320],
          [ 1.0244, -0.8318,  2.6734,  ...,  1.0601,  1.5372,  2.0316],
          [ 1.5146,  0.8250,  0.1793,  ...,  0.6082, -0.9119,  0.3943],
          ...,
          [ 0.0216, -1.5690,  1.2650,  ...,  2.2220,  0.5010,  2.8211],
          [-1.7106,  0.1823, -0.7618,  ...,  0.0459, -0.9736, -0.3225],
          [-0.0267,  0.1126, -0.9643,  ...,  1.1822, -0.0366, -0.1795]],

         [[ 1.0853, -1.6172,  0.1729,  ...,  0.5486, -0.4387,  0.6518],
          [-1.3613,  0.4653,  2.1923,  ..., -0.3632, -0.9289,  1.5911],
          [ 1.7251,  0.2236,  0.2423,  ...,  1.6110,  1.0571,  0.0913],
          ...,
          [ 0.6831,  2.1183,  0.6529,  ..., -1.8377, -1.9571,  2.9264],
          [-0.6733,  0.3157, -0.9592,  ..., -0.8292,  1.1525, -2.6826],
          [-2.0278,  2.3075, -1.1850,  ..., -1.5066,  1.3144, -1.6762]],

         [[ 2.1537,  1.0038, -1.2062,  ..., -1.4361, -0.5517,  1.1747],
          [-0.2129, -1.1561, -

### class torch.nn.Dropout3d(p=0.5, inplace=False)
与前面的类似。

### class torch.nn.AlphaDropout(p=0.5)
在输入上应用 Alpha Dropout。Alpha Dropout 在训练期间, 按照伯努利概率分布, 以概率 p 随机地将输入张量中的部分元素 置进行掩盖, 在每次调用中, 被掩盖的元素是随机的, 并且对输出会进行缩放、变换等操作 以保持均值为 0、标准差为 1。更多信息请参考论文: [Self-Normalizing Neural Networks](https://arxiv.org/abs/1706.02515)。

## <div id = "SparseLayer"></div>Sparse layers

### class torch.nn.Embedding(num_embeddings, embedding_dim, padding_idx=None, max_norm=None, norm_type=2, scale_grad_by_freq=False, sparse=False, _weight=None)
一个保存了固定字典和大小的简单查找表。这个模块常用来保存词嵌入（word embeddings）和用下标检索它们。模块的输入是一个下标的列表，输出是对应的词嵌入。

**参数**
- num_embeddings (int) - 嵌入字典的大小
- embedding_dim (int) - 每个嵌入向量的大小
- padding_idx (int, optional) - 如果提供的话，输出遇到此下标时用零填充
- max_norm (float, optional) - 如果提供的话，会重新归一化词嵌入，使它们的范数小于提供的值
- norm_type (float, optional) - 对于max_norm选项计算p范数时的p
- scale_grad_by_freq (boolean, optional) - 如果提供的话，会根据字典中单词频率缩放梯度

**变量**
- weight (Tensor) -形状为(num_embeddings, embedding_dim)的模块中可学习的权值

**形状**
- 输入： 任意形状的LongTensor，包含了要提取的索引。
- 输出： (*，embedding_dim)，*指的是输入的形状

注意：只有有限数量的优化器支持稀疏梯度。目前有 `optim.SGD`(CUDA和CPU)，`optim.SparseAdam`(CUDA和CPU)，以及`optim.Adagrad`(CPU)。

In [33]:
# an Embedding module containing 10 tensors of size 3
embedding = nn.Embedding(10, 3)
# a batch of 2 samples of 4 indices each
input = torch.LongTensor([[1,2,4,5],[4,3,2,9]])
print(embedding(input))

# example with padding_idx
embedding = nn.Embedding(10, 3, padding_idx=0)
input = torch.LongTensor([[0,2,0,5]])
print(embedding(input))

tensor([[[-0.7935, -1.4070, -0.4081],
         [ 0.6319,  0.4839, -0.6100],
         [-0.6429, -0.9572,  0.7497],
         [-0.7222,  1.0039,  0.8132]],

        [[-0.6429, -0.9572,  0.7497],
         [-0.7251, -1.9815,  1.0354],
         [ 0.6319,  0.4839, -0.6100],
         [ 0.0495,  0.5817,  2.4680]]])
tensor([[[ 0.0000,  0.0000,  0.0000],
         [-0.7291,  0.9612,  0.3609],
         [ 0.0000,  0.0000,  0.0000],
         [ 0.8209, -0.2717,  1.3294]]])


#### classmethod from_pretrained(embeddings, freeze=True)
从给定的2维的FloatTensor创建嵌入实例。

**参数**
- embeddings(Tensor)：包含Embedding权重的FloatTensor。第一维是被传递到Embedding的`num_embeddings`；第二维是`embedding_dim`
- freeze(boolean,optional)：如果为真，则tensor在学习过程中不更新。相当于：embedding.weight.requires_grad = False。

In [34]:
# FloatTensor containing pretrained weights
weight = torch.FloatTensor([[1, 2.3, 3], [4, 5.1, 6.3]])
embedding = nn.Embedding.from_pretrained(weight)
# Get embeddings for index 1
input = torch.LongTensor([1])
print(embedding(input))

tensor([[ 4.0000,  5.1000,  6.3000]])


### class torch.nn.EmbeddingBag(num_embeddings, embedding_dim, max_norm=None, norm_type=2, scale_grad_by_freq=False, mode='mean', sparse=False)
计算一 个’bags’ 里的 embeddings的均值或和, 不用实例化中间的embeddings。

## <div id = "DistanseFunction"></div>Distance functions
### class torch.nn.CosineSimilarity(dim=1, eps=1e-08)
返回沿着 dim 方向计算的 x1 与 x2 之间的余弦相似度。

**Parameters**
- dim (int, optional) – 计算余弦相似度的维度. Default: 1
- eps (float, optional) – 小的值以避免被零除. Default: 1e-8

**Shape**
- Input1: (∗1,D,∗2), 其中的 D 表示 dim 的位置
- Input2: (∗1,D,∗2), 与 Input1 一样的 shape
- Output: (∗1,∗2)

In [35]:
input1 = torch.randn(100, 128)
input2 = torch.randn(100, 128)
cos = nn.CosineSimilarity(dim=1, eps=1e-6)
output = cos(input1, input2)

### class torch.nn.PairwiseDistance(p=2, eps=1e-06, keepdim=False)
计算`batchwise pairwise distance`，向量v1和v2通过计算p范数得到。

**Shape**
- Input1: (N,D), 其中的 D = vector dimension(向量维度)
- Input2: (N,D), 与 Input1 的 shape 一样
- Output: (N)，若keepdim=False，就是(N,1)

In [36]:
pdist = nn.PairwiseDistance(p=2)
input1 = torch.randn(100, 128)
input2 = torch.randn(100, 128)
output = pdist(input1, input2)

## <div id = "LossFunctions"></div>Loss functions
### class torch.nn.L1Loss(size_average=True, reduce=True)
L1损失。逐元素地求出输入x和目标y之间差的绝对值，最后返回绝对值的平均值。即绝对值损失函数

In [37]:
loss = nn.L1Loss()
input = torch.randn(3, 5, requires_grad=True)
target = torch.randn(3, 5)
output = loss(input, target)
output.backward()

### class torch.nn.MSELoss(size_average=True, reduce=True)
输入 x 和 目标 y 之间的均方误差。也就是平方损失函数。

In [38]:
loss = nn.MSELoss()
input = torch.randn(3, 5, requires_grad=True)
target = torch.randn(3, 5)
output = loss(input, target)
output.backward()

### class torch.nn.CrossEntropyLoss(weight=None, size_average=True, ignore_index=-100, reduce=True)
交叉熵损失函数。
### class torch.nn.NLLLoss(weight=None, size_average=True, ignore_index=-100, reduce=True)
负对数似然损失（negative log likelihood loss）。

损失函数还有很多，后续用到再做补充。

## <div id = "VisionLayer"></div> Vision layers

### class torch.nn.PixelShuffle(upscale_factor)
将shape为$[*, r^2C, H, W]$的Tensor重新排列为shape为$[C, rH, rW]$的Tensor。当使用 stride = 1/r 的高效子像素卷积很有用。参见[Real-Time Single Image and Video Super-Resolution Using an Efficient Sub-Pixel Convolutional Neural Network](https://arxiv.org/abs/1609.05158)。

**参数**
- upscale_factor (int) – 增加空间分辨率的因子

**Shape**
- 输入: (N,C∗upscale_factor^2,H,W)
- 输出: (N,C,H∗upscale_factor,W∗upscale_factor)

### class torch.nn.Upsample(size=None, scale_factor=None, mode='nearest', align_corners=None)
对给定的多通道一维时序数据, 二维空间数据, 或三维容积数据进行上采样。

输入数据的格式为 minibatch x channels x depth x height x width. 因此, 对于2-D数据的输入, 期望得到一个4-D张量；对于3-D数据输入, 期望得到一个5-D张量。对3D, 4D, 5D的输入张量进行最近邻、线性、双线性和三线性采样, 可用于该上采样方法。

可以提供 scale_factor 或目标输出的 size 来计算输出的大小。（不能同时都给, 因为这样做是含糊不清的）

**参数**
- size (tuple, optional) – 整型数的元组 ([D_out], [H_out], W_out) 的输出大小
- scale_factor (int / tuple of python:ints, optional) – 图像高度/宽度/深度的乘子
- mode (string, optional) – 上采样算法: nearest | linear | bilinear | trilinear. 默认为: nearest

**Shape**
- 输入: (N,C,Win), (N,C,Hin,Win) 或 (N,C,Din,Hin,Win)
- 输出: (N,C,Wout), (N,C,Hout,Wout) 或 (N,C,Dout,Hout,Wout) 其中: Dout=floor(Din∗scale_factor) 或 size[-3] Hout=floor(Hin∗scale_factor) 或 size[-2] Wout=floor(Win∗scale_factor) 或 size[-1]

In [41]:
input = torch.arange(1, 5).view(1, 1, 2, 2)
print(input)
m = nn.Upsample(scale_factor=2, mode='nearest')
print(m(input))

m = nn.Upsample(scale_factor=2, mode='bilinear')  # align_corners=False
print(m(input))

m = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)
print(m(input))

# Try scaling the same data in a larger tensor
input_3x3 = torch.zeros(3, 3).view(1, 1, 3, 3)
input_3x3[:, :, :2, :2].copy_(input)
print(input_3x3)

m = nn.Upsample(scale_factor=2, mode='bilinear')  # align_corners=False
# Notice that values in top left corner are the same with the small input (except at boundary)
print(m(input_3x3))

m = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)
# Notice that values in top left corner are now changed
print(m(input_3x3))

tensor([[[[ 1.,  2.],
          [ 3.,  4.]]]])
tensor([[[[ 1.,  1.,  2.,  2.],
          [ 1.,  1.,  2.,  2.],
          [ 3.,  3.,  4.,  4.],
          [ 3.,  3.,  4.,  4.]]]])
tensor([[[[ 1.0000,  1.2500,  1.7500,  2.0000],
          [ 1.5000,  1.7500,  2.2500,  2.5000],
          [ 2.5000,  2.7500,  3.2500,  3.5000],
          [ 3.0000,  3.2500,  3.7500,  4.0000]]]])
tensor([[[[ 1.0000,  1.3333,  1.6667,  2.0000],
          [ 1.6667,  2.0000,  2.3333,  2.6667],
          [ 2.3333,  2.6667,  3.0000,  3.3333],
          [ 3.0000,  3.3333,  3.6667,  4.0000]]]])
tensor([[[[ 1.,  2.,  0.],
          [ 3.,  4.,  0.],
          [ 0.,  0.,  0.]]]])
tensor([[[[ 1.0000,  1.2500,  1.7500,  1.5000,  0.5000,  0.0000],
          [ 1.5000,  1.7500,  2.2500,  1.8750,  0.6250,  0.0000],
          [ 2.5000,  2.7500,  3.2500,  2.6250,  0.8750,  0.0000],
          [ 2.2500,  2.4375,  2.8125,  2.2500,  0.7500,  0.0000],
          [ 0.7500,  0.8125,  0.9375,  0.7500,  0.2500,  0.0000],
          [ 0.0000

/home/tina/miniconda3/envs/pytorch/lib/python3.6/site-packages/torch/nn/functional.py:1749: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode))


### class torch.nn.UpsamplingNearest2d(size=None, scale_factor=None)
对多个输入通道组成的输入信号进行2维最近邻上采样。为了指定尺度, 提供了 size 或 scale_factor 作为构造参数。当给定 size, 输出图像的大小为 (h, w)。

**Parameters**
- size (tuple, optional) – 输出图片大小的整型元组(H_out, W_out)
- scale_factor (int, optional) – 图像的 长和宽的乘子。

**Shape**
- Input: (N,C,Hin,Win)
- Output: (N,C,Hout,Wout) 其中 Hout=floor(Hin∗scale_factor) ，Wout=floor(Win∗scale_factor)

In [42]:
input = torch.arange(1, 5).view(1, 1, 2, 2)
print(input)
m = nn.UpsamplingNearest2d(scale_factor=2)
print(m(input))

tensor([[[[ 1.,  2.],
          [ 3.,  4.]]]])
tensor([[[[ 1.,  1.,  2.,  2.],
          [ 1.,  1.,  2.,  2.],
          [ 3.,  3.,  4.,  4.],
          [ 3.,  3.,  4.,  4.]]]])


/home/tina/miniconda3/envs/pytorch/lib/python3.6/site-packages/torch/nn/modules/upsampling.py:173: UserWarning: nn.UpsamplingNearest2d is deprecated. Use nn.Upsample instead.
  warnings.warn("nn.UpsamplingNearest2d is deprecated. Use nn.Upsample instead.")


### class torch.nn.UpsamplingBilinear2d(size=None, scale_factor=None)
对多个输入通道组成的输入信号进行2维双线性上采样。

In [43]:
input = torch.arange(1, 5).view(1, 1, 2, 2)
print(input)
m = nn.UpsamplingBilinear2d(scale_factor=2)
print(m(input))

tensor([[[[ 1.,  2.],
          [ 3.,  4.]]]])
tensor([[[[ 1.0000,  1.3333,  1.6667,  2.0000],
          [ 1.6667,  2.0000,  2.3333,  2.6667],
          [ 2.3333,  2.6667,  3.0000,  3.3333],
          [ 3.0000,  3.3333,  3.6667,  4.0000]]]])


/home/tina/miniconda3/envs/pytorch/lib/python3.6/site-packages/torch/nn/modules/upsampling.py:221: UserWarning: nn.UpsamplingBilinear2d is deprecated. Use nn.Upsample instead.
  warnings.warn("nn.UpsamplingBilinear2d is deprecated. Use nn.Upsample instead.")


## <div id = "Multi-GPU"></div>DataParallel layers (multi-GPU, distributed)

### class torch.nn.DataParallel(module, device_ids=None, output_device=None, dim=0)
在模块级别实现数据并行性。此容器通过将mini-batch划分到不同的设备上来实现给定module的并行。在forward过程中，module会在每个设备上都复制一遍，每个副本都会处理部分输入。在backward过程中，副本上的梯度会累加到原始module上。

batch的大小应该大于所使用的GPU的数量。还应当是GPU个数的整数倍，这样划分出来的每一块都会有相同的样本数量。参见:[Use nn.DataParallel instead of multiprocessing](../介绍/3.Cuda语义.ipynb)

除了Tensor，任何位置参数和关键字参数都可以传到DataParallel中。所有的变量会通过指定的dim来划分（默认值为0）。原始类型将会被广播，但是所有的其它类型都会被浅拷贝。所以如果在模型的forward过程中写入的话，将会被损坏。

**参数说明**
- module – 要被并行的module
- device_ids – CUDA设备，默认为所有设备。
- output_device – 输出设备（默认为device_ids[0]）
```python
net = torch.nn.DataParallel(model, device_ids=[0, 1, 2])
output = net(input_var)
```

### class torch.nn.parallel.DistributedDataParallel(module, device_ids=None, output_device=None, dim=0, broadcast_buffers=True)
在模块级别实现分布式数据并行。此容器通过将mini-batch划分到不同的设备上来实现给定module的并行。module会在每个设备上都复制一遍，每个副本都会处理部分输入。在backward过程中，来自每个节点的梯度被平均。参见:[Use nn.DataParallel instead of multiprocessing](../介绍/3.Cuda语义.ipynb)

这个类的创建要求在进程组模式中已经初始化了分布式包。参见[torch.distributed.init_process_group()](https://pytorch.org/docs/stable/distributed.html#torch.distributed.init_process_group)

```python
torch.distributed.init_process_group(world_size=4, init_method='...')
net = torch.nn.DistributedDataParallel(model)
```

## <div id = "Utilities"></div>Utilities(工具包)
### torch.nn.utils.clip_grad_norm_(parameters, max_norm, norm_type=2)
这个函数是根据参数的范数来衡量的。用于梯度裁剪。范数是对所有梯度进行计算的，等价于把所有输入变量的梯度连接成一个向量，然后对这个向量按范数进行裁剪。梯度将会被原地修改。

**参数**	
- parameters (Iterable[Variable]) – 一个可迭代Tensor, 梯度会被正规化
- max_norm (float or int) – 梯度的最大范数
- norm_type (float or int) – p 范数(指定 p ). 用 'inf' 表示无穷范数

### torch.nn.utils.clip_grad_value_(parameters, clip_value)
以一个特定值裁切一个可迭代参数的梯度。梯度将会被原地修改。
### torch.nn.utils.weight_norm(module, name='weight', dim=0)
将权重标准化应用于给定模块中的指定参数。权重标准化是一个重新参数化的过程，它将权重张量的大小从它的方向中分离出来。该函数会用两个参数通过名字 (e.g.“weight”)代替所指定的参数：一个指定参数的大小 (e.g. “weight_g”), 一个指定参数的方向“weight_v”（）。权重归一化是通过一个hook实现的, 该hook会在forward的每次调用之前根据大小和方向(两个新参数)重新计算权重张量。

默认情况下, dim=0, 范数会在每一个输出的 channel/plane 上分别计算。 若要对整个权重张量计算范数, 使用 dim=None。

参见：https://arxiv.org/abs/1602.07868


**参数**	
- module (nn.Module) – 给定的 module
- name (str, optional) – 权重参数的 name
- dim (int, optional) – 进行范数计算的维度
	

In [ ]:
m = weight_norm(nn.Linear(20, 40), name='weight')
print(m.weight_g.size())
print(m.weight_v.size())

### torch.nn.utils.remove_weight_norm(module, name='weight')
从模块中移除权重归一化再参数化。
```python
m = weight_norm(nn.Linear(20, 40))
remove_weight_norm(m)
```

### PackedSequence
关于RNN部分，待补充。